# Posed Image Dataset Collection

In [ ]:
import os
import shutil
dataset_path = 'dataset/apr24'
seg_path = os.path.join(dataset_path, 'seg')
rgb_path = os.path.join(dataset_path, 'rgb')
depth_path = os.path.join(dataset_path, 'depth')

In [ ]:
from FR3Py.robot.interface import FR3Real
robot = FR3Real()

Test the interface by reading the robot joints:

In [ ]:
robot.getStates()

In [ ]:
from FR3Py.joysticks import PyGameJoyManager
import time    
joy1  = PyGameJoyManager()
joy1.start_daq(joy_idx=0)
joy1.offset_calibration()
time.sleep(3)

In [ ]:
from SimpleHandEye.interfaces.cameras import RealSenseCamera
camera = RealSenseCamera(VGA=False, enable_imu=False, enable_ir=True, emitter_enabled=True, align_to_color=False)
ext_params = camera.getExtrinsics()
int_params = camera.getIntrinsics()

In [ ]:
import pickle
with open(os.path.join(dataset_path,'camera_params.pkl'), 'wb') as f:
    pickle.dump({'ext_params':ext_params, 'int_params':int_params}, f)

In [ ]:
import time
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController
import datetime
import os
import cv2
import pandas as pd
from scipy.spatial.transform import Rotation as R
shutter_old = 0 

if os.path.exists(depth_path):
        shutil.rmtree(depth_path)
# Create the input_model directory
os.makedirs(depth_path, exist_ok=True)

if os.path.exists(rgb_path):
        shutil.rmtree(rgb_path)
# Create the input_model directory
os.makedirs(rgb_path, exist_ok=True)

if os.path.exists(seg_path):
        shutil.rmtree(seg_path)
# Create the input_model directory
os.makedirs(seg_path, exist_ok=True)

def write_depth(depth, filename):
    depth = depth.astype(np.uint16)
    cv2.imwrite(filename, depth)

def write_color(color, filename):
    color = color.astype(np.uint8)
    cv2.imwrite(filename, color)

def write_pose(pose, filename):
    np.savetxt(filename, pose)
    

controller = WaypointController(kp=4)
# Read the initila pose of the robot
time.sleep(1)
state = robot.getStates()

q, dq = state['q'], state['dq']
q = np.hstack([q, np.zeros(2)])
dq = np.hstack([dq, np.zeros(2)])
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']

T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()

x0, y0, z0 = 0.0, 0.0, 0.0 
R0 = np.eye(3)
ef_poses = []
stamps = []
pose_dataset = []
running = 0
while running==0:
    trans = controller.robot.getInfo(q,dq)['P_EE']
    rot = controller.robot.getInfo(q,dq)['R_EE']
    stamps.append(time.time())
    pose = np.vstack([np.hstack([rot, trans.reshape(3,1)]), np.array([0,0,0,1])])

    analog, digital = joy1.read_values()
    running = digital[1]

    cmd = np.array([analog[0],analog[1],analog[2]])
    for i in range(3):
        if np.abs(cmd[i])<0.1:
            cmd[i] = 0 

    if digital[0]==0:
        y0 = y0 - cmd[0]*0.2/100
        x0 = x0 - cmd[1]*0.2/100
        z0 = z0 + cmd[2]*0.2/100
    else:
        omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                              [ cmd[2],   0,      -cmd[0]],
                              [-cmd[1],   cmd[0],      0]])
        R0 = R0@(np.eye(3)+omega_hat/100)
    
    camera.grab_frames()
    cv2.imshow('color', camera.color_frame)
    cv2.waitKey(10)
    # Take a snapshot if required
    shutter = digital[2]
    if shutter==1 and shutter_old==0:
        stamp_ns = int(time.time()*1e9)
        shutter_old = shutter
        depth = camera.depth_frame
        color = camera.color_frame
        print('Taking a picture')
        write_depth(depth, os.path.join(depth_path, f'{stamp_ns}.png'))
        write_color(color, os.path.join(rgb_path, f'{stamp_ns}.png'))
        pos = pose[0:3,3].squeeze()
        q = R.from_matrix(pose[:3,:3]).as_quat().squeeze()
        pose_dataset.append(np.hstack([stamp_ns, pos, q]))
    else:
        shutter_old = shutter

    T= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.getStates()
    q, dq = state['q'], state['dq']
    q = np.hstack([q, np.zeros(2)])
    dq = np.hstack([dq, np.zeros(2)])
    cmd = controller.compute(q,dq, T_cmd=T)
    # robot.setCommands(cmd[:7])
    robot.setCommands(np.zeros(7))

pd.DataFrame(data=pose_dataset, columns=['id', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw']).to_csv(f'{dataset_path}/pose.csv', index=False)
print('Dataset collection procedure ended.')
cv2.destroyAllWindows()

In [ ]:
pd.DataFrame(data=pose_dataset, columns=['id', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw']).to_csv(f'{dataset_path}/pose.csv', index=False)
print('Dataset collection procedure ended.')
cv2.destroyAllWindows()